In this kernel I will be performing some basic yet important parts of a machine learning development pipeline.I build this to refresh my concepts and want to share with anyone who wish the same.
It includes
* Feature Engineering
* Exploratory Data Analysis
* Different Regression model implementaion and comparison

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/vehicle-dataset-from-cardekho/car data.csv")
df.head()

In [ ]:
print(df['Seller_Type'].unique())
print(df['Fuel_Type'].unique())
print(df['Transmission'].unique())
print(df['Owner'].unique())

In [ ]:
df.isnull().sum()


In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
final_dataset=df[['Year','Selling_Price','Present_Price','Kms_Driven','Fuel_Type','Seller_Type','Transmission','Owner']]


In [ ]:
final_dataset['Current Year']=2020


In [ ]:
final_dataset.head()


In [ ]:
final_dataset['no_year']=final_dataset['Current Year']- final_dataset['Year']


In [ ]:
final_dataset.drop(['Year'],axis=1,inplace=True)


In [ ]:
final_dataset.head()


In [ ]:
final_dataset=pd.get_dummies(final_dataset,drop_first=True)


In [ ]:
final_dataset.head()


In [ ]:
final_dataset=final_dataset.drop(['Current Year'],axis=1)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
final_dataset.corr()


In [ ]:
sns.pairplot(final_dataset)



In [ ]:
corrmat = df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
X=final_dataset.iloc[:,1:]
y=final_dataset.iloc[:,0]

In [ ]:
X['Owner'].unique()


In [ ]:
X.head()


# Extra Trees Regression

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
import matplotlib.pyplot as plt
model = ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
print(model.feature_importances_)


In [ ]:
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestRegressor


In [ ]:
regressor=RandomForestRegressor()


In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
print(n_estimators)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
rf = RandomForestRegressor()


In [ ]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(X_train,y_train)


In [ ]:
rf_random.best_params_


In [ ]:
rf_random.best_score_


In [ ]:
predictions=rf_random.predict(X_test)


In [ ]:
sns.distplot(y_test-predictions)


In [ ]:
plt.scatter(y_test,predictions)


In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('random_forest_regression_model.pkl', 'wb')

# dump information to that file
pickle.dump(rf_random, file)

For Deployment of this particular project 
Refer [here](https://github.com/ananyd36/mldeploy)
> Thanks

# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression


In [ ]:
model = LinearRegression()
model.fit(X_train,y_train)
preds = model.predict(X_test)

In [ ]:
sns.distplot(y_test-preds)

In [ ]:
print("MAE:", metrics.mean_absolute_error(y_test,preds))
print("MSE:", metrics.mean_squared_error(y_test,preds))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, preds)))

# Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge


Before Optimization

In [ ]:
model1 = Ridge()
model1.fit(X_train,y_train)
predz = model.predict(X_test)
sns.distplot(y_test-predz)

After Optimization

In [ ]:
alpha = [x for x in np.linspace(start = .5,stop = 1.5,num = 5)]
fit_intercept = ["True","False"]
normalize = ["True","False"]
max_iter = [int(x) for x in np.linspace(start = 800,stop = 1500,num = 5)]
solver = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag']


In [ ]:
param_grid = {
    "alpha":alpha,
    "fit_intercept":fit_intercept,
    "normalize":normalize,
    "max_iter":max_iter,
    'solver':solver
}

In [ ]:
rf = Ridge()

In [ ]:
rf_final = RandomizedSearchCV(estimator = rf,param_distributions = param_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_final.fit(X_train,y_train)

In [ ]:
rf_final.best_params_

In [ ]:
rf_final.best_score_

In [ ]:
preds1 = rf_final.predict(X_test)

In [ ]:
sns.distplot(y_test-preds1)

# Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNet


In [ ]:
model = ElasticNet()
model.fit(X_train,y_train)
preds2 = model.predict(X_test)
sns.distplot(y_test-preds2)

In [ ]:
alpha = [x for x in np.linspace(start = .5,stop = 1.5,num = 5)]
l1_ratio = [x for x in np.linspace(start= 0.2,stop = 0.9,num = 5)]
fit_intercept = ["True","False"]
normalize = ["True","False"]
max_iter = [int(x) for x in np.linspace(start = 800,stop = 1500,num = 5)]
selection = ['cyclic','random']

In [ ]:
param_grid = {
    "alpha":alpha,
    "l1_ratio":l1_ratio,
    "fit_intercept":fit_intercept,
    "normalize":normalize,
    "max_iter":max_iter,
    "selection":selection
}

In [ ]:
rf = ElasticNet()

In [ ]:
rf_final2 = RandomizedSearchCV(estimator = rf,param_distributions = param_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_final2.fit(X_train,y_train)

In [ ]:
rf_final2.best_params_

In [ ]:
rf_final2.best_score_

In [ ]:
preds = rf_final2.predict(X_test)
sns.distplot(y_test-preds)

Not a good approach!!

# Xgboost

In [ ]:
import xgboost as xgb

model = xgb.XGBRegressor()
model.fit(X_train,y_train)
pred3 = model.predict(X_test)
sns.distplot(y_test-pred3)

In [ ]:
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error,mean_absolute_error
print(mean_squared_error(y_test,pred3))
print("MAE", mean_absolute_error(y_test,pred3))

In [ ]:
sns.scatterplot(y_test,pred3)

Hyperparameter Tuning

In [ ]:
from scipy.stats import uniform, randint

xgb_model = xgb.XGBRegressor()

params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, n_iter=200, cv=3, verbose=1, n_jobs=1, return_train_score=True)

search.fit(X_train, y_train)

# report_best_scores(search.cv_results_, 1)

In [ ]:
search.best_params_

In [ ]:
search.best_score_

In [ ]:
predss = search.predict(X_test)
sns.distplot(y_test-predss)

In [ ]:
print("MAE:", metrics.mean_absolute_error(y_test,predss))
print("MSE:", metrics.mean_squared_error(y_test,predss))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predss)))

In [ ]:
sns.scatterplot(y_test,predss)